<a href="https://colab.research.google.com/github/ChiliJoe/genai-intro/blob/main/ChatGPT_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an expansion of tutorial from https://youtu.be/0-zlUy7VUjg using real-world API from openweathermap.org.

Let's use the openweathermap current weather API that requires geo code as input.

In [1]:
%%capture
# Set openweathermap.org API key
%env OWM_API_KEY=

# Set OpenAI API key
%env OPENAI_API_KEY=

In [2]:
%%capture
!rm -rf helper
!git clone https://github.com/ChiliJoe/genai-intro.git helper

from helper.weather import function_list

# Vanilla OpenAI

In [3]:
%%capture
!pip install openai

In [9]:
from IPython.display import display, Markdown
from helper.demolib import Color

import openai
import os
import json

openai.api_key  = os.getenv('OPENAI_API_KEY')

class Chat:
  def __init__(self, system_message, function_list):
    self.messages = [{"role": "system", "content": system_message}]
    self.function_list = function_list
    self.function_descriptions = [f["description"] for f in function_list.values()]

  def ask(self, role_content, role="user", function_name=None):
    if role == "function":
      self.messages.append({"role": role, "name": function_name, "content": role_content})
    else:
      self.messages.append({"role": role, "content": role_content})

    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=self.messages,
        functions=self.function_descriptions,
        function_call="auto"
    )
    ai_message = res.choices[0].message
    self.messages.append(ai_message)

    # process function call
    if ai_message["content"] is None and ai_message["function_call"]:
      target_function_name = ai_message["function_call"]["name"]
      target_function = self.function_list[target_function_name]["function"]
      func_params = json.loads(ai_message['function_call']['arguments'])

      print(Color.cyan(f"Invoking {target_function_name} with parameters {func_params}"))
      function_response = target_function(**func_params)
      print(Color.white("Function response: "), end="")
      print(Color.magenta(f"{function_response}"))

      self.ask(role_content=json.dumps(function_response),
               role="function",
               function_name=target_function_name
               )

    elif ai_message["content"]:
      print(Color.green("===== Final response: ====="))
      display(Markdown(ai_message["content"]))

In [10]:
sysmessage="""
You are a helpful AI chatbot that responds to user in the style of The Terminator.
"""
chat = Chat(sysmessage, function_list)

In [11]:
chat.ask("What's the weather like in the city where Barack Obama was born?")

Invoking get_location_data with parameters {'city': 'Honolulu', 'country': 'US'}
Function response: [{'name': 'Honolulu', 'local_names': {'mi': 'Honoruru', 'ru': 'Гонолулу', 'de': 'Honolulu', 'lt': 'Honolulu', 'pl': 'Honolulu', 'ko': '호놀룰루', 'fa': 'هونولولو', 'ta': 'ஹொனலுலு', 'es': 'Honolulu', 'ur': 'ہونولولو', 'ks': 'ہونولولو', 'zh': '檀香山 / 火奴鲁鲁', 'ar': 'هونولولو', 'en': 'Honolulu', 'ja': 'ホノルル', 'el': 'Χονολουλού', 'uk': 'Гонолулу'}, 'lat': 21.304547, 'lon': -157.855676, 'country': 'US', 'state': 'Hawaii'}]
Invoking get_current_weather with parameters {'lat': 21.304547, 'lon': -157.855676, 'unit': 'metric'}
Function response: {'coord': {'lon': -157.8557, 'lat': 21.3045}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 23.97, 'feels_like': 24.62, 'temp_min': 21.63, 'temp_max': 26.1, 'pressure': 1017, 'humidity': 84}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 350}, 'clouds': {'all': 75}, 'dt': 168

In Honolulu, the city where Barack Obama was born, the current weather is as follows:

- Temperature: 23.97°C
- Description: Broken clouds
- Wind speed: 2.06 m/s

Please note that the weather conditions may change, so it's always a good idea to check for updates.